<a href="https://colab.research.google.com/github/enjoyPG/2023Gifted/blob/main/students/joon0425/20230722/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam
from torchvision import transforms
import torchvision.datasets as ds
from torchvision.models.vgg import vgg16
from tqdm import tqdm
from torchvision.transforms.functional import to_pil_image

# 1. CNN 개요

CNN은 합성곱과 신경망층으로 구성돼있다.

합성곱을 통해 지역적인 요소를 압축하여, 차원을 축소시키면서 특성을 추출해낸다.

이후 그 특성을 신경망층을 통해 분류, 예측 등의 문제를 해결할 수 있다.

이 예제에서는 분류를 위해 신경망층과 교차 엔트로피 함수를 통해 n개의 클래스에 대한 각각의 확률을 구한다.

# 2. 합성곱 기본 블럭 정의

기본 블럭에서는 in channel의 텐서의 크기를 유지하면서 hidden channel, out channel 로 합성곱한 뒤, max pooling을 진행하여 크기를 줄인다.

여기서는 max pooling을 통해 크기가 가로 세로 각 절반으로 줄어든다.  
  ex ) (24x24) -> (12x12)

In [5]:
# inc -> hdim -> outc 채널로 Conv를 함

class BasicBlock(nn.Module):
  def __init__(self,inc,outc,hdim):
    super(BasicBlock,self).__init__()
    self.conv1 = nn.Conv2d(inc,hdim,kernel_size=3,padding=1)
    self.conv2 = nn.Conv2d(hdim,outc,kernel_size=3,padding=1)
    self.relu = nn.ReLU()
    self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
  def forward(self,x):
    return self.pool(self.relu(self.conv2(self.relu(self.conv1(x)))))

# 3. CNN 모델 정의

CNN 모델은 기본 블럭 여러 개와 FC layer(전결합 신경망층)으로 구성된다.

이 예제에서는

(3, 32, 32) -> (32, 16, 16) -> (128, 8, 8) -> (256, 4, 4) 로 합성곱이 진행되고,

(256, 4, 4) -> (4096) 으로 flatten한 뒤.

(4096) -> (2048) -> (256) -> (10) 으로 전결합 신경망층을 구성한다.

활성화 함수로는 ReLU를 사용했다.

In [6]:
class cnn(nn.Module):
  def __init__(self,nClass):
    super(cnn,self).__init__()

    # 합성곱 기본 블록
    self.block1 = BasicBlock(inc=3,outc=32,hdim=16)
    self.block2 = BasicBlock(inc=32,outc=128,hdim=64)
    self.block3 = BasicBlock(inc=128, outc=256,hdim=128)
    # 합성곱 이후 신경망
    self.fc1 = nn.Linear(in_features=4096,out_features=2048)
    self.fc2 = nn.Linear(in_features=2048,out_features=256)
    self.fc3 = nn.Linear(in_features=256,out_features=nClass)

    self.relu = nn.ReLU()
  def forward(self,x):
    x = torch.flatten(self.block3(self.block2(self.block1(x))),start_dim=1) # 합성곱 후 flatten
    return self.fc3(self.relu(self.fc2(self.relu(self.fc1(x))))) # 신경망

# 4. 데이터 전처리

RandomCrop으로 4만큼 패딩 후 (36, 36) -> (32, 32)로 자른다.

RandomHorizontalFlip에서는 50% 확률로 좌우로 반전시킨다.

이후 텐서로 변환시키고, 이를 정규화한다.

In [7]:
transform = transforms.Compose([
    transforms.RandomCrop((32,32),padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914,0.4822,0.4465),std=(0.247,0.243,0.261))
])

# 5. 데이터 불러오기

데이터를 불러오고, 전처리를 적용한 뒤, 데이터로더에 데이터를 넣어준다.

그리고 모델을 불러온다.

In [19]:
training_data = ds.CIFAR10(root="./",train=True,download=True,transform=transform)
test_data = ds.CIFAR10(root="./",train=False,download=True,transform=transform)

train_loader = DataLoader(training_data,batch_size=32,shuffle=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = cnn(nClass=10)
model.to(device)

Files already downloaded and verified
Files already downloaded and verified


cnn(
  (block1): BasicBlock(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): BasicBlock(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=256,

# 6. 모델 학습

모델의 예측값을 받고, 손실함수로 손실을 계산한 뒤, 이를 오류 역전파한다.

100 epochs에 대해 batch size 32로 미니배치 학습을 진행한다. (batch size는 위 셀에서 데이터로더에 정의됨)

In [24]:
lr = 0.001
epochs = 100
loss_func = nn.CrossEntropyLoss()
optim = Adam(model.parameters(),lr=lr)
for epoch in range(epochs):
  for data,label in train_loader:
    optim.zero_grad()
    y = model(data.to(device))
    loss = loss_func(y,label.to(device))
    loss.backward()
    optim.step()
  if epoch==0 or epoch%10==9:
    print(f"epoch{epoch+1}, loss:{loss.item()}")
torch.save(model.state_dict(),"CIFAR.pth")

KeyboardInterrupt: ignored

# 7. 사전 학습된 모델 불러오기

사전 학습된 vgg16 모델을 불러오고, 전결합 신경망 층을 정의한다.

모델의 구조는 아래 출력을 참조하라.

In [28]:
model = vgg16(pretrained=True)
fc = nn.Sequential(
    nn.Linear(512*7*7,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,10),
)
model.classifier = fc
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:09<00:00, 56.7MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 8. 모델 학습

위의 모델 학습 부분과 일치한다.

tqdm을 통해 progress bar를 추가했다는 점 외엔 차이점이 없다.

In [33]:
lr = 0.0001
loss_func = nn.CrossEntropyLoss()
optim = Adam(model.parameters(),lr=lr)
for epoch in range(30):
  iterator = tqdm(train_loader)
  for data,label in iterator:
    optim.zero_grad()
    y = model(data.to(device))
    loss = loss_func(y,label.to(device))
    loss.backward()
    optim.step()
    iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")
torch.save(model.state_dict(),"CIFAR_pretrained.pth")

cpu


  0%|          | 0/1563 [00:03<?, ?it/s]


KeyboardInterrupt: ignored

# 9. 정확도 구하기

예측값과 라벨이 얼마나 일치하는지 test loader의 데이터를 통해서 구한다.

In [ ]:
model.load_state_dict(torch.load("CIFAR_pretrained.pth"),map_location=device)
num_corr=0
with torch.no_grad():
  for data,label in test_loader:
    output = model(data.to(device))
    y = output.data.max(1)[1]
    corr = y.eq(label.to(device).data).sum().item()
    num_corr += corr
  print(f"Accuracy : {num_corr/len(test_data)}")
